## Establish a connection to the XRP Publicy Ledger

## Manage keys and wallets
## `xrpl.wallet`
* Use the xrpl.wallet module to create a wallet from a given seed or or via a Testnet faucet.
* To create a wallet from a seed (in this case, the value generated using xrpl.keypairs):

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from xrpl.clients import JsonRpcClient
JSON_RPC_URL = "https://s.altnet.rippletest.net:51234"
client = JsonRpcClient(JSON_RPC_URL)

In [4]:
# Create a wallet using the testnet faucet:
# https://xrpl.org/xrp-testnet-faucet.html
from xrpl.wallet import generate_faucet_wallet
test_wallet = generate_faucet_wallet(client, debug=True)

Attempting to fund address rENT2aXJ6vVjGb2toVyw3M8ZVwKLMenfUm
Faucet fund successful.


## To create a wallet from a Testnet faucet:

In [6]:
test_wallet = generate_faucet_wallet(client)
test_account = test_wallet.classic_address
print("Classic address:", test_account)

Classic address: rK1DbsArZVU85HVxuNUGDqttcEjpHhgVD5


## xrpl.core.keypairs
* Use the xrpl.core.keypairs module to generate seeds and derive keypairs and addresses from those seed values.

In [7]:
from xrpl.core import keypairs
seed = keypairs.generate_seed()
public, private = keypairs.derive_keypair(seed)
test_account = keypairs.derive_classic_address(public)
print("Here's the public key:")
print(public)
print("Here's the private key:")
print(private)
print("Store this in a secure place!")

Here's the public key:
EDA1443B62734E1AFF7DBD312EF0CD9249DD82D9D96E84FDF5818F0ACB438A1246
Here's the private key:
ED87E38B6AA453BDC784E28C651A58F7FDDD308E9E9C46073C659F1C139728AE60
Store this in a secure place!


# Serialize and sign transactions

* First serialize data from JSON and other formats into the XRP Ledger's canonical format
* Then to authorize the transaction by digitally signing it with the account's private key. The xrpl-py library provides several methods to simplify this process.

In [8]:
from xrpl.models.transactions import Payment
from xrpl.transaction import safe_sign_transaction, send_reliable_submission
from xrpl.ledger import get_latest_validated_ledger_sequence
from xrpl.account import get_next_valid_seq_number

current_validated_ledger = get_latest_validated_ledger_sequence(client)
test_wallet.sequence = get_next_valid_seq_number(test_wallet.classic_address, client)

# prepare the transaction
# the amount is expressed in drops, not XRP
# see https://xrpl.org/basic-data-types.html#specifying-currency-amounts
my_tx_payment = Payment(
    account=test_wallet.classic_address,
    amount="2200000",
    destination="rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe",
    last_ledger_sequence=current_validated_ledger + 20,
    sequence=test_wallet.sequence,
    fee="10",
)
# sign the transaction
my_tx_payment_signed = safe_sign_transaction(my_tx_payment,test_wallet)

# submit the transaction
tx_response = send_reliable_submission(my_tx_payment_signed, client)

# Get fee from the XRP Ledger


In [9]:
# use the get_fee function
from xrpl.ledger import get_fee
fee = get_fee(client)
print(fee)
# 10

10


In [14]:
from xrpl.models.transactions import Payment
from xrpl.transaction import send_reliable_submission, safe_sign_and_autofill_transaction
# prepare the transaction
# the amount is expressed in drops, not XRP
# see https://xrpl.org/basic-data-types.html#specifying-currency-amounts
my_tx_payment = Payment(
    account=test_wallet.classic_address,
    amount="2200000",
    destination="rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe"
)

# sign the transaction with the autofill method
# (this will auto-populate the fee, sequence, and last_ledger_sequence)
my_tx_payment_signed = safe_sign_and_autofill_transaction(my_tx_payment, test_wallet, client)
print(my_tx_payment_signed)
# Payment(
#     account='rMPUKmzmDWEX1tQhzQ8oGFNfAEhnWNFwz',
#     transaction_type=<TransactionType.PAYMENT: 'Payment'>,
#     fee='10',
#     sequence=16034065,
#     account_txn_id=None,
#     flags=0,
#     last_ledger_sequence=10268600,
#     memos=None,
#     signers=None,
#     source_tag=None,
#     signing_pub_key='EDD9540FA398915F0BCBD6E65579C03BE5424836CB68B7EB1D6573F2382156B444',
#     txn_signature='938FB22AE7FE76CF26FD11F8F97668E175DFAABD2977BCA397233117E7E1C4A1E39681091CC4D6DF21403682803AB54CC21DC4FA2F6848811DEE10FFEF74D809',
#     amount='2200000',
#     destination='rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe',
#     destination_tag=None,
#     invoice_id=None,
#     paths=None,
 #    send_max=None,
 #    deliver_min=None
# )

# submit the transaction
tx_response = send_reliable_submission(my_tx_payment_signed, client)

Payment(account='rK1DbsArZVU85HVxuNUGDqttcEjpHhgVD5', transaction_type=<TransactionType.PAYMENT: 'Payment'>, fee='10', sequence=27019112, account_txn_id=None, flags=0, last_ledger_sequence=27019315, memos=None, signers=None, source_tag=None, signing_pub_key='EDB5A7485329FF8833F5259A016D6D448FCFC35AAA2B189560910BBAEAE4D0E0EC', txn_signature='718C19B2A3E2087BF06AE2B3761ECDD2F096470C21AF8D8E6E6F4080EC50B473CDF6AEDD033186F8265EA4B83CC743D5A1998E2ADF16395B03EE4AFD7C8BAF0F', amount='2200000', destination='rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe', destination_tag=None, invoice_id=None, paths=None, send_max=None, deliver_min=None)


# Subscribe to XRP ledger updates


In [ ]:
# Send subscribe and unsubscribe requests using the WebSocket Network Client.
from xrpl.clients import WebsocketClient
url = "wss://s.altnet.rippletest.net/"
from xrpl.models.requests import Subscribe, StreamParameter
req = Subscribe(streams=[StreamParameter.LEDGER])
# NOTE: this code will run forever without a timeout, until the process is killed
with WebsocketClient(url) as client:
    client.send(req)
    for message in client:
        print(message)
# {'result': {'fee_base': 10, 'fee_ref': 10, 'ledger_hash': '7CD50477F23FF158B430772D8E82A961376A7B40E13C695AA849811EDF66C5C0', 'ledger_index': 18183504, 'ledger_time': 676412962, 'reserve_base': 20000000, 'reserve_inc': 5000000, 'validated_ledgers': '17469391-18183504'}, 'status': 'success', 'type': 'response'}
# {'fee_base': 10, 'fee_ref': 10, 'ledger_hash': 'BAA743DABD168BD434804416C8087B7BDEF7E6D7EAD412B9102281DD83B10D00', 'ledger_index': 18183505, 'ledger_time': 676412970, 'reserve_base': 20000000, 'reserve_inc': 5000000, 'txn_count': 0, 'type': 'ledgerClosed', 'validated_ledgers': '17469391-18183505'}
# {'fee_base': 10, 'fee_ref': 10, 'ledger_hash': 'D8227DAF8F745AE3F907B251D40B4081E019D013ABC23B68C0B1431DBADA1A46', 'ledger_index': 18183506, 'ledger_time': 676412971, 'reserve_base': 20000000, 'reserve_inc': 5000000, 'txn_count': 0, 'type': 'ledgerClosed', 'validated_ledgers': '17469391-18183506'}
# {'fee_base': 10, 'fee_ref': 10, 'ledger_hash': 'CFC412B6DDB9A402662832A781C23F0F2E842EAE6CFC539FEEB287318092C0DE', 'ledger_index': 18183507, 'ledger_time': 676412972, 'reserve_base': 20000000, 'reserve_inc': 5000000, 'txn_count': 0, 'type': 'ledgerClosed', 'validated_ledgers': '17469391-18183507'}

{'result': {'fee_base': 10, 'fee_ref': 10, 'ledger_hash': '0F0CB0B658B7FC91E62887864DD60CFC3F1E0EF220BB43707ADFAECEF8FE5099', 'ledger_index': 27019307, 'ledger_time': 703541692, 'reserve_base': 10000000, 'reserve_inc': 2000000, 'validated_ledgers': '25432110-27019307'}, 'status': 'success', 'type': 'response'}
{'fee_base': 10, 'fee_ref': 10, 'ledger_hash': 'B676BF7AC7C2487C2D1196CA9CD15092EAF1F5AE968EE1E1227D076EDD992338', 'ledger_index': 27019308, 'ledger_time': 703541700, 'reserve_base': 10000000, 'reserve_inc': 2000000, 'txn_count': 0, 'type': 'ledgerClosed', 'validated_ledgers': '25432110-27019308'}
{'fee_base': 10, 'fee_ref': 10, 'ledger_hash': '6A3195F78B6CA26505113F91D19AC2B4F4A54F7E40130D6C99120ADCD5953D3F', 'ledger_index': 27019309, 'ledger_time': 703541701, 'reserve_base': 10000000, 'reserve_inc': 2000000, 'txn_count': 0, 'type': 'ledgerClosed', 'validated_ledgers': '25432110-27019309'}
{'fee_base': 10, 'fee_ref': 10, 'ledger_hash': 'D7682AF91FDFEF137AA633F1B9D2A2F615E3B268A8